# Sans2d data reduction 

In [ ]:
import scipp as sc
import sciline
import scippneutron as scn
import esssans as sans
from esssans.types import *

## Define reduction parameters

In [ ]:
params = {}
params[NeXusMonitorName[Incident]] = 'monitor2'
params[NeXusMonitorName[Transmission]] = 'monitor4'
band = sc.linspace('wavelength', 2.0, 16.0, num=2, unit='angstrom')
params[WavelengthBands] = band
params[WavelengthBins] = sc.linspace(
    'wavelength', start=band[0], stop=band[-1], num=141
)
params[WavelengthMask] = sc.DataArray(
    data=sc.array(dims=['wavelength'], values=[True]),
    coords={
        'wavelength': sc.array(
            dims=['wavelength'], values=[2.21, 2.59], unit='angstrom'
        )
    },
)

params[QBins] = sc.linspace(dim='Q', start=0.01, stop=0.6, num=141, unit='1/angstrom')
params[Filename[BackgroundRun]] = 'SANS2D00063159.hdf5'
params[Filename[SampleRun]] = 'SANS2D00063114.hdf5'
params[Filename[DirectRun]] = 'SANS2D00063091.hdf5'
params[DirectBeamFilename] = 'DIRECT_SANS2D_REAR_34327_4m_8mm_16Feb16.hdf5'
params[BeamCenter] = sc.vector(value=[0.0945643, -0.082074, 0.0], unit='m')
params[NonBackgroundWavelengthRange] = sc.array(
    dims=['wavelength'], values=[0.7, 17.1], unit='angstrom'
)
params[CorrectForGravity] = True

## Create pipeline using Sciline

We use all providers available in `esssans` as well as the `sans2d`-specific providers, which include I/O and mask setup specific to the [Sans2d](https://www.isis.stfc.ac.uk/Pages/sans2d.aspx) instrument:

In [ ]:
providers = sans.providers + sans.sans2d.providers
pipeline = sciline.Pipeline(providers, params=params)

## Use the pipeline

### Compute final result

We can get the graph for computing the background-subtracted $I(Q)$:

In [ ]:
iofq = pipeline.get(BackgroundSubtractedIofQ)

Before we compute the result, we can visualize the pipeline:

In [ ]:
# left-right layout works better for this graph
iofq.visualize(graph_attr={'size': '20,10', 'rankdir': 'LR'})

Now we can compute the result:

In [ ]:
result = iofq.compute()
result.plot()

### Compute intermediate results

In [ ]:
results = iofq.compute((IofQ[SampleRun], IofQ[BackgroundRun]))
results[IofQ[BackgroundRun]].hist().plot()

In [ ]:
scn.instrument_view(pipeline.compute(MaskedData[SampleRun]).hist(), pixel_size=0.0075)

In [ ]:
scn.instrument_view(
    pipeline.compute(Clean[SampleRun, Numerator]).value.hist(), pixel_size=0.0075
)

In [ ]:
scn.instrument_view(pipeline.compute(SolidAngle[SampleRun]), pixel_size=0.0075)

In [ ]:
denominator = pipeline.get(Clean[SampleRun, Denominator]).compute()
denominator.value.sum('spectrum').plot(norm='log')

In [ ]:
pipeline.compute(IofQ[SampleRun]).hist().plot()
# sc.plot( sc.collapse(pipeline.compute(IofQ[SampleRun]).hist(), keep='Q'))

In [ ]:
pipeline.visualize(IofQ[SampleRun], graph_attr={'size': '12,10'})
pipeline.visualize(IofQ[BackgroundRun], graph_attr={'size': '12,10'})
pipeline.visualize(
    BackgroundSubtractedIofQ, graph_attr={'size': '20,10', 'rankdir': 'TD'}
)

In [ ]:
pipeline.get(CleanMonitor[SampleRun, Incident]).visualize()

In [ ]:
pipeline.compute(CleanMonitor[SampleRun, Incident]).value

In [ ]:
pipeline.get(MaskedData[SampleRun]).visualize()

We now plot them on the same figure to asses the level of background noise

In [ ]:
sc.plot(monitors, norm='log', grid=True)

From this, we define a wavelength range between 0.7 &#8491; and 17.1 &#8491; where data is not considered to be background.

In [ ]:
scn.instrument_view(sample.hist(), pixel_size=0.0075)

## Beam center finder

The beam is not guaranteed to travel through the center of the detector panel,
and we thus have to apply a horizontal and vertical offset to our pixel positions so that the beam centre is at `x = y = 0`.
This is necessary for subsequent azimuthal averaging of the data counts into $Q$ bins.

The `beam_center` utility in the `sans` module is designed for this.
It requires us to define a $Q$ range over which convergence will be checked.

In [ ]:
q_range = sc.linspace('Q', 0.02, 0.3, 71, unit='1/angstrom')

center = sans.beam_center(
    data=sample,
    data_monitors=sample_monitors,
    direct_monitors=direct_monitors,
    wavelength_bins=wavelength_bins,
    q_bins=q_range,
    gravity=gravity,
)
print(center)

# Now shift pixels positions to get the correct beam center
sample.coords['position'] -= center